In [121]:
import pandas as pd
import numpy as np

class Data_Handler:
    
    def __init__(self):
        self.dflisting = None
        self.dfprice = None
        self.dfmerge = None
        
    def getDatas(self):
        print("Récupération des données")
        listing = pd.read_csv("https://storage.googleapis.com/h3-data/listings_final.csv", sep=";")
        prices = pd.read_csv("https://storage.googleapis.com/h3-data/price_availability.csv", sep=";")
        self.dflisting, self.dfprice = listing, prices
        return "data loaded \nFiles : \n dflisting {} \n dfprice{}".format(listing.shape,prices.shape)
        
    def groupDatas(self):
        print("Fusion des données")
        moyenne = self.dfprice.groupby('listing_id')['local_price'].mean()
        self.dfmerge = pd.merge(moyenne, self.dflisting, on="listing_id")
        print ("Données fusionnées")
        
    def getAndGroup(self):
        self.getDatas()
        self.groupDatas()
        print("Données récupérer et grouper")
        

In [122]:
data = Data_Handler()
dfmerge = data.getAndGroup()
%time

Récupération des données
Fusion des données
Données fusionnées
Données récupérer et grouper
Wall time: 0 ns


In [123]:
class FeatureRecipe :
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.varcat = []
        self.varfloat = []
        self.varint = []
    
    def dropnan(self, threshold: float):
        print("supprimer les colonnes avec NaN")
        drop=[]
        for col in self.data.columns:
            a = self.data[col]
            if a.isna().sum() / self.data.shape[0] >= threshold:
                drop.append(col)
        self.data.drop(columns=drop, inplace = True)
        print("Supprimer les colonnes avec un taux de NaN égal ou supérieur ou égale -> thresold")
        
    def dropuseless(self):
        print("Supprimer les colonnes inutiles")
        drop = ["Unnamed: 0","name","neighborhood","latitude","longitude"]
        self.data.drop(axis="columns", labels=drop, inplace=True)
    
    def getduplicate(self):
        duplicates = []
        for col in range(self.data.shape[1]-1):
            for cols in range(col+1,self.data.shape[1]-1):
                print("{} {}".format(col,cols))
                if sum(np.where(self.data.iloc[:,[col]]==self.data.iloc[:,[cols]],0,1)) == 0:
                    duplicates.append(cols)
        return duplicates
    
    def dropduplicate(self):
        print("nombre de colonnes avant suppresion {}".format(self.data.columns.size))
        test = []
        i = 1
        for col in self.data.columns:
            a = self.data[col]
            for cols in self.data.columns[i:]:
                b = self.data[cols]
                if col == cols and a.equals(b):
                    test.append(cols)
                    break
            i+=1
            
        self.data.drop(columns=test, inplace=True)
        print("nombre de colonnes après suppression {}".format(self.data.columns.size))
    
    def separatevariables(self):
        for column in self.data.columns:
            if self.data[column].dtypes == float:
                self.varfloat.append(self.data[column])
            elif self.data[column].dtypes == int:
                self.varint.append(self.data[column])
            else:
                self.varcat.append(self.data[column])
    
    def RecipeDatas(self, threshold: float):
        self.dropnan(threshold)
        self.dropuseless()
        self.dropduplicate()
        self.separatevariables()
        

In [124]:
data.dfmerge.head()

,listing_id,local_price,Unnamed: 0,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
0,56093,170.000000,12,Beau duplex dans le Marais,entire_home,Paris,3e arrondissement,48.867284,2.358431,4,2,1,1.0,False,False,True,True,False,0.88,1.0
1,57207,49.952756,13,Belle Chambre pour court,private_room,Paris,Vaugirard,48.846184,2.304455,2,1,1,1.0,False,False,True,False,False,0.87,1.0
2,114543,107.374026,19,Charming 1bdr 55m² - Eiffel Tower,entire_home,Paris,NaN,48.849530,2.290219,2,1,1,1.0,False,False,True,True,False,0.90,0.9
3,149534,169.000000,9,GREAT WARM FULL APT LE HAUT MARAIS,entire_home,Paris,NaN,48.866360,2.361844,4,2,1,1.0,False,False,True,True,False,1.00,0.4
4,164255,75.876209,28,Perfect place in Le Marais - Paris,entire_home,Paris,3e arrondissement,48.861398,2.364299,4,2,1,1.0,False,False,True,False,False,1.00,1.0


In [125]:
dfrecipe = FeatureRecipe(data.dfmerge)

In [126]:
dfrecipe.RecipeDatas(5)

supprimer les colonnes avec NaN
Supprimer les colonnes avec un taux de NaN égal ou supérieur ou égale -> thresold
Supprimer les colonnes inutiles
nombre de colonnes avant suppresion 15
nombre de colonnes après suppression 15


In [129]:
data.dfmerge.head()

,listing_id,local_price,type,city,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
0,56093,170.000000,entire_home,Paris,4,2,1,1.0,False,False,True,True,False,0.88,1.0
1,57207,49.952756,private_room,Paris,2,1,1,1.0,False,False,True,False,False,0.87,1.0
2,114543,107.374026,entire_home,Paris,2,1,1,1.0,False,False,True,True,False,0.90,0.9
3,149534,169.000000,entire_home,Paris,4,2,1,1.0,False,False,True,True,False,1.00,0.4
4,164255,75.876209,entire_home,Paris,4,2,1,1.0,False,False,True,False,False,1.00,1.0


In [ ]:
class FeatureExtractor:
    
    def __init__(self, data: pd.DataFrame, dflist: list):
        self.data = data
        self.dflist = dflist
        self.X_train = None
        self.Y_train = None
        self.X_test = None
        self.Y_test = None
        
    def extract(self):
        for col in self.data.columns:
            if col in self.dflist:
        self.data.drop(col, axis="columns", inplace=True)
    
    def split(self, size_test: float, rng: int, target: pd.Series):
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(self.data.loc[:,self.data.columns != target], target, test_size=size_test,)
        return self.X_train, self.X_test, self.Y_train, self.Y_test
    
    def extract_split(self):
        self.extract()
        self.split()

In [ ]:
class ModelBuilder:
    """
        Class for train and print results of ml model 
    """
    def __init__(self, model_path: str = None, save: bool = None):
        pass
    def __repr__(self):
        pass
    def predict(self, X) -> np.ndarray:
        pass
    def save_model(self, path:str):
        #with the format : 'model_{}_{}'.format(date)
        pass
    def load_model(self):
        try:
            #load model
            pass
        except:
            pass